In [1]:
import numpy as np

In [27]:
a = np.ones((3,3,2,2,2))

In [28]:
a[0,1,0,1,0] = 100

In [29]:
b = a + np.arange(9).reshape((3,3,1,1,1))

In [30]:
b[:,:,0,1,0]

array([[  1., 101.,   3.],
       [  4.,   5.,   6.],
       [  7.,   8.,   9.]])

In [38]:
b.shape

(3, 3, 2, 2, 2)

In [37]:
b.mean(axis=(2,3,4))

array([[ 1.   , 14.375,  3.   ],
       [ 4.   ,  5.   ,  6.   ],
       [ 7.   ,  8.   ,  9.   ]])

In [ ]:
# test changed operator due to global variable
const = 10
f = lambda x : x + const
const = 10000
f(2)

In [24]:
# test adding new dim
tmp[:,:,np.newaxis,np.newaxis,np.newaxis].shape

(3, 3, 1, 1, 1)

In [2]:
i, j = 0, 1

In [5]:
[i,j] == [1,0]

False

In [13]:
# Function to compute the curl of a vector field component
def compute_curl(U, V, W):
    curl_x = np.gradient(W, axis=1) - np.gradient(V, axis=2)
    curl_y = np.gradient(U, axis=2) - np.gradient(W, axis=0)
    curl_z = np.gradient(V, axis=0) - np.gradient(U, axis=1)
    return np.array([curl_x, curl_y, curl_z])

def curl_F(F): # F in shape [3,3,nx,ny,nz]
    # Initialize an array to store the curl of the tensor field
    curl_F = np.zeros((3, 3, *F.shape[2:]))

    # Compute the curl for each component of the stress tensor
    for i in range(3):
        for j in range(3):
            curl_F[i, j] = compute_curl(F[(i+1)%3, (j+1)%3], F[(i+2)%3, (j+1)%3], F[(i+2)%3, (j+2)%3])

    # curl_F is the resulting field
    return curl_F

In [59]:
e_ijk = lambda i,j,k: (i-j)*(j-k)*(k-i)/2
F_jk_i = lambda F,j,k,i: np.gradient(F[j,k,:,:,:],axis=i)
curl_F_mk = lambda F,m,k: np.sum([e_ijk(i,j,m)*F_jk_i(F,j,k,i) for i,j in itertools.product(range(3),repeat=2)],axis=0)
curl_F = lambda F: np.array([curl_F_mk(F,m,k) for m,k in itertools.product(range(3),repeat=2)]).reshape(F.shape)

try to use DAMASK grid_filters to check consistency

In [65]:
cF = curl_F(F_rand).shape


In [66]:
np.linalg.norm(cF)

9.643650760992955

In [67]:
G_x = lambda F_rand: G(F_rand).reshape([3,3,*shape])
GGG_F_rand = G_x(G_x(G_x(G_x(G_x(F_rand)))))
c_GGG_F = curl_F(GGG_F_rand)
np.linalg.norm(c_GGG_F)

8.189518702692673

In [70]:
GFr = G_x(F_rand)
cGFr = curl_F(GFr)
np.linalg.norm(cGFr)

8.189518702692672

In [24]:
np.reshape(G_F_rand,[3,3,*shape]).shape

(3, 3, 5, 5, 5)

In [17]:
# Function to compute the 2D curl-like operation for a component
def compute_curl_2d(U, V):
    curl = np.gradient(V, axis=2) - np.gradient(U, axis=3)
    return curl

def curl_F_2(F):
    # Initialize an array to store the curl of the tensor field
    curl_F = np.zeros((3, 3, *F.shape[2:]))

    # Compute the curl for each component of the stress tensor
    # Note: The last dimension is squeezed since nz = 1
    for i in range(3):
        for j in range(3):
            U = F[(i+1)%3, (j+1)%3, :, :, 0]
            V = F[(i+2)%3, (j+1)%3, :, :, 0]
            curl_F[i, j, :, :, 0] = compute_curl_2d(U, V)

    # curl_F is the resulting field
    return curl_F

In [20]:
tmp_2 = ddot42(Ghat4, tmp_1)
tmp_2.shape

(3, 3, 3, 3, 1)

In [ ]:
q